In [ ]:
# This Python 3 environment comes with many helpful analytics libraries installed
# It is defined by the kaggle/python Docker image: https://github.com/kaggle/docker-python
# For example, here's several helpful packages to load

import numpy as np # linear algebra
import pandas as pd # data processing, CSV file I/O (e.g. pd.read_csv)

# Input data files are available in the read-only "../input/" directory
# For example, running this (by clicking run or pressing Shift+Enter) will list all files under the input directory

import os
for dirname, _, filenames in os.walk('/kaggle/input'):
    for filename in filenames:
        print(os.path.join(dirname, filename))

# You can write up to 20GB to the current directory (/kaggle/working/) that gets preserved as output when you create a version using "Save & Run All" 
# You can also write temporary files to /kaggle/temp/, but they won't be saved outside of the current session

In [ ]:
import pandas as pd
df = pd.read_csv('/kaggle/input/acea-water-prediction/Aquifer_Auser.csv',na_values=['NULL','null', 'nan','NaN'])
df

In [ ]:
df['Date'] = pd.to_datetime(df['Date'],format='%d/%m/%Y')
df

In [ ]:
# controllare le differenze di tempo
shift = df['Date'] - df['Date'].shift(1)
shift

In [ ]:
shift.value_counts()

In [ ]:
df = df.sort_values(by='Date')
df

In [ ]:
df.columns

In [ ]:
import matplotlib.pyplot as plt
import seaborn as sns

#df = df.reset_index()

f, ax = plt.subplots(nrows=len(df.columns.to_list())-3, ncols=1, figsize=(22, 160))

for idx,x in enumerate(df.columns):
    
    if x not in ['Date','index','level_0']:
        sns.lineplot(x=df.Date, y=df[x], ax=ax[idx-3], color='dodgerblue')
        ax[idx-3].set_title(x, fontsize=14)
        ax[idx-3].set_ylabel(ylabel=x, fontsize=14)


plt.show()

In [ ]:
import math
def calc(x):
    if x == 0 or pd.isna(x):
        return -100
    else:
        return x
        

old = df.Volume_POL.copy()
old = old.apply(lambda x: calc(x) )

f, ax = plt.subplots(nrows=2, ncols=1, figsize=(15, 5))

sns.lineplot(x=df.Date, y=old, ax=ax[0], color='dodgerblue')
ax[0].set_title('Feature: Volume_POL', fontsize=14)
ax[0].set_ylabel(ylabel='Volume_POL', fontsize=14)

plt.show()

In [ ]:
df[ (df['Volume_POL'] != 0) & (df['Volume_POL'].isnull() == False) ]
df[ (df['Rainfall_Gallicano'] != 0) & (df['Rainfall_Gallicano'].isnull() == False) ]

In [ ]:
df2 = df.copy()
df2 = df.set_index('Date')
#df.reset_index(inplace=True)
df2 = df2.Volume_POL.interpolate(method='time', axis=0).bfill()
#df['Volume_POL'].interpolate(method='linear',inplace=True).ffill().bfill()
df2

In [ ]:


old = df2
f, ax = plt.subplots(nrows=2, ncols=1, figsize=(15, 5))

sns.lineplot(x=df.index, y=old, ax=ax[0], color='dodgerblue')
ax[0].set_title('Feature: Volume_POL', fontsize=14)
ax[0].set_ylabel(ylabel='Volume_POL', fontsize=14)

plt.show()

In [ ]:
df = df[ (df['Rainfall_Gallicano'] != 0) & (df['Rainfall_Gallicano'].isnull() == False) ]
df

**Check stationary features**

In [ ]:
import matplotlib.pyplot as plt
import seaborn as sns


rolling_window = 52
f, ax = plt.subplots(nrows=len(df.columns.to_list()), ncols=1, figsize=(22, 160))

for idx,x in enumerate(df.columns):
    if x != 'Date':
        sns.lineplot(x=df.Date, y=df[x], ax=ax[idx], color='indianred')
        sns.lineplot(x=df.Date, y=df[x].rolling(rolling_window).mean(), ax=ax[idx], color='black', label='rolling mean')
        sns.lineplot(x=df.Date, y=df[x].rolling(rolling_window).std(), ax=ax[idx], color='blue', label='rolling std')
        ax[idx].set_title(x, fontsize=14)
        ax[idx].set_ylabel(ylabel=x, fontsize=14)

plt.show()

**Check correlation**

In [ ]:
f, ax = plt.subplots(nrows=1, ncols=1, figsize=(100, 108))

original_cols = df.columns

corrmat = df[original_cols].corr()

#sns.heatmap(corrmat, annot=True, vmin=-1, vmax=1, cmap='coolwarm_r', ax=ax[0])
#ax[0].set_title('Correlation Matrix of Original Features', fontsize=16)
sns.heatmap(corrmat, annot=True, vmin=-1, vmax=1, cmap='coolwarm_r', ax=ax)
ax.set_title('Correlation Matrix of Original Features', fontsize=16)

#shifted_cols = [ 'Depth_to_Groundwater_seasonal', 
               # 'Temperature_seasonal_shift_r_2M',
               # 'Drainage_Volume_seasonal_shift_1M', 'River_Hydrometry_seasonal_shift_3M']
#corrmat = df[shifted_cols].corr()

#sns.heatmap(corrmat, annot=True, vmin=-1, vmax=1, cmap='coolwarm_r', ax=ax[1])
#ax[1].set_title('Correlation Matrix of Shifted Features', fontsize=16)


plt.tight_layout()
plt.show()

In [ ]:
import matplotlib.pyplot as plt
import seaborn as sns
import numpy as np

ts_diff = np.diff(df.Temperature_Orentano)
df['Temperature_Orentano_diff_1'] = np.append([0], ts_diff)
ts_diff = np.diff(df.Temperature_Orentano_diff_1)
df['Temperature_Orentano_diff_2'] = np.append([0], ts_diff)

ts_diff = np.diff(df.Temperature_Monte_Serra)
df['Temperature_Monte_Serra_diff_1'] = np.append([0], ts_diff)
ts_diff = np.diff(df.Temperature_Monte_Serra_diff_1)
df['Temperature_Monte_Serra_diff_2'] = np.append([0], ts_diff)

ts_diff = np.diff(df.Temperature_Ponte_a_Moriano)
df['Temperature_Ponte_a_Moriano_diff_1'] = np.append([0], ts_diff)
ts_diff = np.diff(df.Temperature_Ponte_a_Moriano_diff_1)
df['Temperature_Ponte_a_Moriano_diff_2'] = np.append([0], ts_diff)

ts_diff = np.diff(df.Temperature_Lucca_Orto_Botanico)
df['Temperature_Lucca_Orto_Botanico_diff_1'] = np.append([0], ts_diff)
ts_diff = np.diff(df.Temperature_Lucca_Orto_Botanico_diff_1)
df['Temperature_Lucca_Orto_Botanico_diff_2'] = np.append([0], ts_diff)

ts_diff = np.diff(df.Volume_POL)
df['Volume_POL_diff_1'] = np.append([0], ts_diff)
ts_diff = np.diff(df.Volume_POL_diff_1)
df['Volume_POL_diff_2'] = np.append([0], ts_diff)

ts_diff = np.diff(df.Volume_CC1)
df['Volume_CC1_diff_1'] = np.append([0], ts_diff)
ts_diff = np.diff(df.Volume_CC1_diff_1)
df['Volume_CC1_diff_2'] = np.append([0], ts_diff)

ts_diff = np.diff(df.Volume_CC2)
df['Volume_CC2_diff_1'] = np.append([0], ts_diff)
ts_diff = np.diff(df.Volume_CC2_diff_1)
df['Volume_CC2_diff_2'] = np.append([0], ts_diff)

ts_diff = np.diff(df.Volume_CSA)
df['Volume_CSA_diff_1'] = np.append([0], ts_diff)
ts_diff = np.diff(df.Volume_CSA_diff_1)
df['Volume_CSA_diff_2'] = np.append([0], ts_diff)

ts_diff = np.diff(df.Volume_CSAL)
df['Volume_CSAL_diff_1'] = np.append([0], ts_diff)
ts_diff = np.diff(df.Volume_CSAL_diff_1)
df['Volume_CSAL_diff_2'] = np.append([0], ts_diff)

ts_diff = np.diff(df.Volume_CSAL)
df['Volume_CSAL_diff_1'] = np.append([0], ts_diff)
ts_diff = np.diff(df.Volume_CSAL_diff_1)
df['Volume_CSAL_diff_2'] = np.append([0], ts_diff)

ts_diff = np.diff(df.Hydrometry_Monte_S_Quirico)
df['Hydrometry_Monte_S_Quirico_diff_1'] = np.append([0], ts_diff)
ts_diff = np.diff(df.Hydrometry_Monte_S_Quirico_diff_1)
df['Hydrometry_Monte_S_Quirico_diff_2'] = np.append([0], ts_diff)

ts_diff = np.diff(df.Hydrometry_Piaggione)
df['Hydrometry_Piaggione_diff_1'] = np.append([0], ts_diff)
ts_diff = np.diff(df.Hydrometry_Piaggione_diff_1)
df['Hydrometry_Piaggione_diff_2'] = np.append([0], ts_diff)



In [ ]:
import matplotlib.pyplot as plt
import seaborn as sns
import numpy as np

f, ax = plt.subplots(nrows=1, ncols=1, figsize=(100, 108))

original_cols = ['Rainfall_Gallicano', 'Rainfall_Pontetetto', 'Rainfall_Monte_Serra',
       'Rainfall_Orentano', 'Rainfall_Borgo_a_Mozzano', 'Rainfall_Piaggione',
       'Rainfall_Calavorno', 'Rainfall_Croce_Arcana',
       'Rainfall_Tereglio_Coreglia_Antelminelli',
       'Rainfall_Fabbriche_di_Vallico','Depth_to_Groundwater_LT2', 
       'Temperature_Orentano_diff_2',
       'Temperature_Monte_Serra_diff_2', 
       'Temperature_Ponte_a_Moriano_diff_2',
       'Temperature_Lucca_Orto_Botanico_diff_2', 
       'Volume_POL_diff_2', 'Volume_CC1_diff_2',
       'Volume_CC2_diff_2', 'Volume_CSA_diff_2', 'Volume_CSAL_diff_2', 
       'Hydrometry_Monte_S_Quirico_diff_2',
       'Hydrometry_Piaggione_diff_2']

corrmat = df[original_cols].corr()

sns.heatmap(corrmat, annot=True, vmin=-1, vmax=1, cmap='coolwarm_r', ax=ax)
ax.set_title('Correlation Matrix of Original Features', fontsize=16)


plt.tight_layout()
plt.show()

In [ ]:
from sklearn.model_selection import TimeSeriesSplit

N_SPLITS = 3

X = df.Date
y = df.Depth_to_Groundwater_LT2

folds = TimeSeriesSplit(n_splits=N_SPLITS)

In [ ]:
import matplotlib.pyplot as plt
import seaborn as sns
from datetime import datetime, date


f, ax = plt.subplots(nrows=N_SPLITS, ncols=2, figsize=(16, 9))

for i, (train_index, valid_index) in enumerate(folds.split(X)):
    X_train, X_valid = X[train_index], X[valid_index]
    y_train, y_valid = y[train_index], y[valid_index]

    sns.lineplot(x= X_train, y= y_train, ax=ax[i,0], color='dodgerblue', label='train')
    sns.lineplot(x= X_train[len(X_train) - len(X_valid):(len(X_train) - len(X_valid) + len(X_valid))], 
                 y= y_train[len(X_train) - len(X_valid):(len(X_train) - len(X_valid) + len(X_valid))], 
                 ax=ax[i,1], color='dodgerblue', label='train')

    for j in range(2):
        sns.lineplot(x= X_valid, y= y_valid, ax=ax[i, j], color='darkorange', label='validation')
    ax[i, 0].set_title(f"Rolling Window with Adjusting Training Size (Split {i+1})", fontsize=16)
    ax[i, 1].set_title(f"Rolling Window with Constant Training Size (Split {i+1})", fontsize=16)

for i in range(N_SPLITS):
    ax[i, 0].set_xlim([date(2006, 1, 1), date(2019, 6, 30)])
    ax[i, 1].set_xlim([date(2006, 1, 1), date(2019, 6, 30)])
plt.tight_layout()
plt.show()

In [ ]:
for i, (train_index, valid_index) in enumerate(folds.split(X)):
    #print(i)
    if i == 0:
        X_train, X_valid = X[train_index], X[valid_index]
        y_train, y_valid = y[train_index], y[valid_index]

X_train, X_valid

In [ ]:
#df = df.set_index('Date')

df[df.index.isin(X_train)]


In [ ]:
from datetime import datetime, date

#df = df.set_index('Date')

target = 'Depth_to_Groundwater_LT2'
features = [feature for feature in df.columns if feature != target]

X = df[df.index < datetime.strptime('2018-12-31', '%Y-%m-%d')][features]
y = df[df.index < datetime.strptime('2018-12-31', '%Y-%m-%d')][target]


X_test = df[df.index > datetime.strptime('2018-12-31', '%Y-%m-%d')][features]#.reset_index(drop=True)
y_test = df[df.index > datetime.strptime('2018-12-31', '%Y-%m-%d')][target]#.reset_index(drop=True)

                       
X

In [ ]:
from fbprophet import Prophet



#X = X.reset_index()
X_prop = X.copy()
y_prop = y.copy()

X_prop = X_prop[['Date','Rainfall_Gallicano', 
'Rainfall_Pontetetto', 
'Rainfall_Monte_Serra',
'Rainfall_Orentano', 
'Rainfall_Borgo_a_Mozzano',
'Rainfall_Calavorno', 
'Rainfall_Croce_Arcana',
'Rainfall_Tereglio_Coreglia_Antelminelli',
'Rainfall_Fabbriche_di_Vallico', 
'Temperature_Orentano_diff_2', 
'Temperature_Monte_Serra_diff_2', 
'Temperature_Ponte_a_Moriano_diff_2', 
'Temperature_Lucca_Orto_Botanico_diff_2',  
'Volume_POL_diff_2',  
'Volume_CC1_diff_2',
'Volume_CC2_diff_2',  
'Volume_CSA_diff_2',  
'Volume_CSAL_diff_2',
'Hydrometry_Monte_S_Quirico_diff_2']]

X_prop=X_prop.rename(columns={'Date':'ds'})
X_prop['y'] = y_prop

print(X_prop.columns)

In [ ]:
cols = ['Rainfall_Gallicano', 
'Rainfall_Pontetetto', 
'Rainfall_Monte_Serra',
'Rainfall_Orentano', 
'Rainfall_Borgo_a_Mozzano',
'Rainfall_Calavorno', 
'Rainfall_Croce_Arcana',
'Rainfall_Tereglio_Coreglia_Antelminelli',
'Rainfall_Fabbriche_di_Vallico', 
'Temperature_Orentano_diff_2', 
'Temperature_Monte_Serra_diff_2', 
'Temperature_Ponte_a_Moriano_diff_2', 
'Temperature_Lucca_Orto_Botanico_diff_2',  
'Volume_POL_diff_2',  
'Volume_CC1_diff_2',
'Volume_CC2_diff_2',  
'Volume_CSA_diff_2',  
'Volume_CSAL_diff_2',
'Hydrometry_Monte_S_Quirico_diff_2']
for x in cols:
    nan_rows = df[df[x].isnull()]
    print(len(nan_rows))


In [ ]:
np.where(y.isnull() == True)
X.iloc[np.where(y.isnull() == True)[0],:]

In [ ]:
mean_val = X_prop.Rainfall_Monte_Serra.mean()
X_prop['Rainfall_Monte_Serra']=X_prop.Rainfall_Monte_Serra.fillna(mean_val)

mean_val = X_prop.Hydrometry_Monte_S_Quirico_diff_2.mean()
X_prop['Hydrometry_Monte_S_Quirico_diff_2']=X_prop.Hydrometry_Monte_S_Quirico_diff_2.fillna(mean_val)

In [ ]:
y.interpolate().ffill()
X_prop['y'] = y.values
np.where(X_prop['y'].isnull() == True)

In [ ]:
cols = ['Rainfall_Gallicano', 
'Rainfall_Pontetetto', 
'Rainfall_Monte_Serra',
'Rainfall_Orentano', 
'Rainfall_Borgo_a_Mozzano',
'Rainfall_Calavorno', 
'Rainfall_Croce_Arcana',
'Rainfall_Tereglio_Coreglia_Antelminelli',
'Rainfall_Fabbriche_di_Vallico', 
'Temperature_Orentano_diff_2', 
'Temperature_Monte_Serra_diff_2', 
'Temperature_Ponte_a_Moriano_diff_2', 
'Temperature_Lucca_Orto_Botanico_diff_2',  
'Volume_POL_diff_2',  
'Volume_CC1_diff_2',
'Volume_CC2_diff_2',  
'Volume_CSA_diff_2',  
'Volume_CSAL_diff_2',
'Hydrometry_Monte_S_Quirico_diff_2']

for x in cols:
    nan_rows = X_prop[X_prop[x].isnull()]
    print(len(nan_rows))
    
m = Prophet()
for x in cols:
    m.add_regressor(x)

m.fit(X_prop)

In [ ]:
# X_TEST:


mean_val = X_test.Hydrometry_Monte_S_Quirico_diff_2.mean()
X_test['Hydrometry_Monte_S_Quirico_diff_2']=X_test.Hydrometry_Monte_S_Quirico_diff_2.fillna(mean_val)



cols = ['Rainfall_Gallicano', 
'Rainfall_Pontetetto', 
'Rainfall_Monte_Serra',
'Rainfall_Orentano', 
'Rainfall_Borgo_a_Mozzano',
'Rainfall_Calavorno', 
'Rainfall_Croce_Arcana',
'Rainfall_Tereglio_Coreglia_Antelminelli',
'Rainfall_Fabbriche_di_Vallico', 
'Temperature_Orentano_diff_2', 
'Temperature_Monte_Serra_diff_2', 
'Temperature_Ponte_a_Moriano_diff_2', 
'Temperature_Lucca_Orto_Botanico_diff_2',  
'Volume_POL_diff_2',  
'Volume_CC1_diff_2',
'Volume_CC2_diff_2',  
'Volume_CSA_diff_2',  
'Volume_CSAL_diff_2',
'Hydrometry_Monte_S_Quirico_diff_2']

for x in cols:
    nan_rows = X_test[X_test[x].isnull()]
    print(len(nan_rows))


X_test = X_test[['Date','Rainfall_Gallicano', 
'Rainfall_Pontetetto', 
'Rainfall_Monte_Serra',
'Rainfall_Orentano', 
'Rainfall_Borgo_a_Mozzano',
'Rainfall_Calavorno', 
'Rainfall_Croce_Arcana',
'Rainfall_Tereglio_Coreglia_Antelminelli',
'Rainfall_Fabbriche_di_Vallico', 
'Temperature_Orentano_diff_2', 
'Temperature_Monte_Serra_diff_2', 
'Temperature_Ponte_a_Moriano_diff_2', 
'Temperature_Lucca_Orto_Botanico_diff_2',  
'Volume_POL_diff_2',  
'Volume_CC1_diff_2',
'Volume_CC2_diff_2',  
'Volume_CSA_diff_2',  
'Volume_CSAL_diff_2',
'Hydrometry_Monte_S_Quirico_diff_2']]

X_test=X_test.rename(columns={'Date':'ds'})
X_test

In [ ]:
future = m.make_future_dataframe(periods=365)
future

In [ ]:
X_prop.set_index('ds',inplace=True)

In [ ]:
X_test.set_index('ds',inplace=True)

In [ ]:
def set_temp(ds,column):
    date = (pd.to_datetime(ds)).date()
    print(ds)
    if X_prop[date:].empty:
        return X_test[date:][column].values[0]
    else:
        return (X_prop[date:][column]).values[0]
    
    return 0

future['Rainfall_Gallicano'] = future['ds'].apply(lambda x : set_temp(x,'Rainfall_Gallicano'))
future['Rainfall_Pontetetto']  = future['ds'].apply(lambda x : set_temp(x,'Rainfall_Pontetetto'))
future['Rainfall_Monte_Serra'] = future['ds'].apply(lambda x : set_temp(x,'Rainfall_Monte_Serra'))
future['Rainfall_Orentano']  = future['ds'].apply(lambda x : set_temp(x,'Rainfall_Orentano'))
future['Rainfall_Borgo_a_Mozzano'] = future['ds'].apply(lambda x : set_temp(x,'Rainfall_Borgo_a_Mozzano')) 
future['Rainfall_Calavorno'] = future['ds'].apply(lambda x : set_temp(x,'Rainfall_Calavorno'))
future['Rainfall_Croce_Arcana']  = future['ds'].apply(lambda x : set_temp(x,'Rainfall_Croce_Arcana'))
future['Rainfall_Tereglio_Coreglia_Antelminelli'] = future['ds'].apply(lambda x : set_temp(x,'Rainfall_Tereglio_Coreglia_Antelminelli'))
future['Rainfall_Fabbriche_di_Vallico']  = future['ds'].apply(lambda x : set_temp(x,'Rainfall_Fabbriche_di_Vallico')) 
future['Temperature_Orentano_diff_2']  = future['ds'].apply(lambda x : set_temp(x,'Temperature_Orentano_diff_2'))
future['Temperature_Monte_Serra_diff_2']  = future['ds'].apply(lambda x : set_temp(x,'Temperature_Monte_Serra_diff_2'))
future['Temperature_Ponte_a_Moriano_diff_2'] = future['ds'].apply(lambda x : set_temp(x,'Temperature_Ponte_a_Moriano_diff_2'))
future['Temperature_Lucca_Orto_Botanico_diff_2'] = future['ds'].apply(lambda x : set_temp(x,'Temperature_Lucca_Orto_Botanico_diff_2'))
future['Volume_POL_diff_2'] = future['ds'].apply(lambda x : set_temp(x,'Volume_POL_diff_2'))
future['Volume_CC1_diff_2'] = future['ds'].apply(lambda x : set_temp(x,'Volume_CC1_diff_2'))
future['Volume_CC2_diff_2'] = future['ds'].apply(lambda x : set_temp(x,'Volume_CC2_diff_2'))
future['Volume_CSA_diff_2'] = future['ds'].apply(lambda x : set_temp(x,'Volume_CSA_diff_2'))
future['Volume_CSAL_diff_2'] = future['ds'].apply(lambda x : set_temp(x,'Volume_CSAL_diff_2'))
future['Hydrometry_Monte_S_Quirico_diff_2'] = future['ds'].apply(lambda x : set_temp(x,'Hydrometry_Monte_S_Quirico_diff_2'))

forecast = m.predict(future)


forecast[['ds', 'yhat', 'yhat_lower', 'yhat_upper']].tail()


In [ ]:
fig1 = m.plot(forecast)
fig2 = m.plot_components(forecast)
